# 九、密钥交换基础
## 可信第三方
假设通信网络中有n个用户，他们相互通信需要彼此之间的通信密钥，这样每位用户需要储存O(N)个密钥。

一个相对更好的解决方案是，网络中存在一个可信第三方（TTP，Online Trusted 3rd Party），**存储有每位用户的密钥**，两个用户需要通信时，根据这两位用户的密
钥计算出通信密钥发送给双方。

设想这样一个简单的可信第三方协议：A想和B通信，于是向TTP发送和B通信的请求，第三方用A的密钥，对通信密钥AB进行加密，同时用B的密钥对通信密钥AB进行加密（>成为Ticket），将两条信息发送给A，A再将Ticket发送给B。这样A、B用各自的密钥都可以解密出通信密钥AB。

这个协议仅能防范监听者，无法防范主动攻击，比如最为简单的重复攻击。密码学的中心定理指出了，任何通过可信第三方实现的加密系统，都可以不需要第三方。

## Merkel难题
设想一个没有可信第三方的协议，假设依然是攻击者只能监听，不具备主动攻击的能力。Merkel提出了一个可以达到该目的，但相对效率较低的模型。

运用Merkel难题，难题（Puzzles）被定义为可以解，但相对比较难解的问题。

假设E是一个对称加密系统，密钥的长度为128位。选用P作为密钥，其中P的前96位为0，后32位是随机数。用P对特定信息进行加密，这成为一个难题（Puzzle）。对方需>要穷举232种可能性来寻找对应的P。

通信过程中，A向B发送攻击2<sup>32</sup>个难题，每个难题都是一个特定的P对“特定信息||密钥”的加密。B在接受到的难题当中，任选一个进行穷举破解，得到“特定信
息||密钥”，将特定信息（标识符）发送给A。A通过特定信息（标识符）查找到该信息对应的通信密钥。

这个过程中，A和B的工作量均为O(N)，而监听者破解所需要的工作量为O(N<sup>2</sup>)，这套系统具备平方优势（quatratic gap）。

可以证明，如果加密系统是一个完全的黑箱，那么平方优势是最优解。但Merkel难题的问题在于效率过低，因此在现实中完全没有被采用。

## 迪菲－赫尔曼协议（The Diffie-Hellman protocol）
同样假定攻击者只有监听能力，不能抵御主动攻击。

给定一个十分大的素数p（比如长度达到600位），一个取值范围在[1, p]之间的整数g。A从[1, p-1]中随机抽取一个整数a，计算A = g<sup>a</sup> mod p，同样B也从[1, p-1]中随机抽取一个整数b，计算B = g<sup>b</sup> mod p。建立通信时，A向B发送信息A，而B向A发送信息B。

共享密钥定义为g<sup>ab</sup> mod p = B<sup>a</sup> mod p = A<sup>b</sup> mod p。双方都可以计算获得该共享密钥。

监听者仅仅知道A和B，要计算g<sup>ab<sup> mod p的话，设定p的长度为n，其难度为exp(O(∛n))。

## 公钥加密（Public-key encryption）
公钥加密系统包括三个算法：1）生成器G()，生成一对公钥（pk）和私钥（sk）。2）加密算法E(pk, m)，使用公钥进行加密。3）解密算法D(sk, c)，使用私钥进行解密>。同一组信息，在公钥加密私钥解密的过程中能够还原。

这个系统可以用来发送通信密钥x。首先A向B发送公钥，B使用公钥加密通信密钥x发送给A，A使用私钥解密后，得到通信密钥x。
同样，公钥加密系统也不能防范主动攻击。需要进一步构造。

# 十、数论简介
## 标记符号
密钥交换协议、电子签名、公钥加密都需要用到数论的相关知识。

N被记为正整数，p表示素数。Z<sub>N</sub>表示从0到N-1的整数集合。在这个集合中，可以进行加法和乘法的模除。
整数x和y的最大公约数记做gcd(x, y)。对任意整数x和y，存在一对整数a和b，使得ax + by = gcd(x, y)。如果x和y的最大公约数为1，则称x和y是互素数。

如果在集合中，xy = 1，那么称x和y互为模逆元。

当且仅当x和N是互素数的时候，x在集合N当中有模逆元。

集合Z<sub>N</sub><sup>'\*'</sup>是集合Z_N中所有具有模逆元的元素的集合。推论：如果p为质数，那么Z<sub>N</sub><sup>'\*'</sup>就是集合Z<sub>N</sub>去掉0>。在星号集合N中，要寻找一个元素的模逆元，可以使用扩展欧几里得算法，其时间复杂度是O(log<sup>2</sup>N)。

这样，在集合上就可以求解方程ax + b = 0，这里x = -b * a<sup>-1</sup>。

## 费马和欧拉
费马小定理：对于素数的星号集合p，对星号集合中的任意元素x，有在集合P中计算x<sup>p-1</sup> = 1。也即x * x<sup>p-2</sup> = 1，即集合P中，x-1 = x<sup>p-2</sup>。这一计算模逆元的办法只使用于素数集合，并且效率也不如扩展欧几里得算法。

费马小定理可以用于寻找素数，假定要生成1024位长的素数，首先，在集合[2<sup>1024</sup>, 2<sup>1025</sup>-1]上随机选取一个整数p，在集合p上测试2<sup>p-1</sup>是否等于1，若等于，则p为素数，否则继续抽取。这是一个生成素数的简单算法，但非最优算法。

欧拉定理：素数p的星号集合是可以写成一个循环群，即存在一个元素g，使得星号集合可以写成{1, g, g<sup>2</sup>, g<sup>3</sup>, …, g<sup>p-2</sup>}，注意到>根据费马小定理，继续迭代下去g<sup>p-1</sup>恰好等于1。g被称为星号集合的迭代器。同时，并不是星号集合中的所有元素都是迭代器。

对星号集合中的任意元素x，可以生成一个集合{1, g, g<sup>2</sup>, g<sup>3</sup>, …}，集合中的所有元素均在星号集合中，该集合中的元素数量成为秩，记ord<sub>p</sub>(g)。拉格朗日定理指出，ord<sub>p</sub>(g)可以整除p-1。

星号集合中元素的数量记为φ(N)，对于星号集合中的任意元素x，在集合N上有x<sup>(φ(N))</sup>=1。

## 模e次方根
有了模逆元的定义，在集合N上，可以计算ax + b = 0，解是x = -b * a<sup>-1</sup>。接下来要解决的问题是如何解平方根，立方根，高次方根。这些根可能存在，也>可能不存在。

先考虑在一个素数集合中。若e和p-1是互素数，则星号集合中的所有元素均有e次方根。

考虑p是一个奇数，e=2的情况，显然p-1和2不是互素数。具有平方根的元素称为二次剩余。当p是奇素数时，集合上的二次剩余数量为(p – 1) / 2 + 1。

欧拉定理：如果x在关于p的星号集合上的二次剩余，那么在关于p的集合中，x<sup>(p-1)/2</sup> = 1。计算其平方根分为两种情况：1）p = 3 mod 4，那么平方根为x<sup>(p + 1)/4</sup>。2）p = 1 mod 4，大约在O(log<sup>3</sup>p)的时间内可以解出。

当要在一个合数的集合上求解时，问题则复杂得多，需要首先对N进行因子分解了。

## 算术算法
加减的时间复杂度是O(N)；乘法朴素算法为O(N<sup>2</sup>)，可以缩减到O(N<sup>1.585</sup>)；除法为O(N<sup>2</sup>)。

幂运算做以下类似处理：g<sup>53</sup> = g<sup>32</sup> * g<sup>16</sup> * g<sup>4</sup> * g<sup>1</sup>。这叫做重复平方算法，可以有效降低时间消耗。

## 难解的问题
模幂运算的逆运算：求解离散对数。迪菲•赫尔曼协议的破解，就需要利用求解离散对数。

合数相关问题：1）因子分解。2）多项式求解。
